In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker, relationship, join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance
from aquabyte.data_access_utils import DataAccessUtils
import datetime as dt
from dateutil import parser

import pickle
from PIL import Image, ImageDraw

pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 50)

<h1> Establish connection to database and perform query for base dataset </h1>

In [ ]:
# AWS credentials
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")

data_access_utils = DataAccessUtils()

# prod SQL credentaials
sql_credentials = json.load(open(os.environ["PROD_SQL_CREDENTIALS"]))

sql_query = '''
select * from keypoint_annotations
where captured_at >= '2019-05-15'
and site_id = 23
and pen_id = 4;
'''

original_df = data_access_utils.extract_from_database(sql_query)
# original_df = original_df.loc[:, ~original_df.columns.duplicated()]
        

In [ ]:
regex = '/root/data/s3/test/**/*.jpg'
files = glob.glob(regex, recursive=True)
raw_files = sorted([f for f in raw_files if 'resize' in f])



In [ ]:
stereo_frame_pairs = []
for raw_file in raw_files:
    timestamp = raw_file.split('/')[-2].replace('at=', '')
    ts = parser.parse(timestamp)
    